# Analysis of seismic catalogues  
this program will prepare and plot seismicity in map and section view  
Marco Scuderi 

### Import the packages needed

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime 

### Import and manipulate the dataset

In [ ]:
aq=pd.read_csv('Aquila2009.csv', sep=';')  
aq

### Now we need to work on formatting the time column
Pandas **to_datetime** will convert your string representation of a date to an actual date format that can be recognized by Pandas and used in time series. 

In [ ]:
aq['time'] = pd.to_datetime(aq['origin_time'])
aq

In [ ]:
# to keep the dataframe clean we can delete the column of origin_time
aq=aq.drop(['origin_time'],axis=1)
aq

In [ ]:
#we can also rename columns for clarity
aq.rename(columns= {'id_dd':'id'}, inplace=True)
aq

### Work on coordinate system for the projection

Build the [UTM](https://en.wikipedia.org/wiki/Universal_Transverse_Mercator_coordinate_system) system, we chose the utm33 

In [ ]:
import pyproj
utm33 = pyproj.Proj(proj='utm',zone=33, ellps='WGS84', preserve_units=False)
#utm33 = pyproj.Proj("+proj=utm +zone=33, +north +ellps=WGS84 +datum=WGS84 +units=m +no_defs")

Convert the coordinate system from lat,lon,depth in UTM x,y,z

In [ ]:
aq['utmx'],aq['utmy']=utm33(np.array(aq['lon']),np.array(aq['lat']))   #lat e lon di tutto il dataset in utm33 
aq['utmx'] = aq['utmx']/1000   # tranform m to km
aq['utmy'] = aq['utmy']/1000
aq

We want to individuate the mainshock and use it as a reference point for the analysis we will do later

In [ ]:
#Referene earthquake (mainshock) that is the center of the section here we use the mainshock
mainev_ref = aq.loc[np.argmax(aq.mag)]
mainev_ref


Now we have all we need to **plot all the seismic sequence on a map**

In [ ]:
%matplotlib qt
#%matplotlib inline
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111)

ax.scatter(aq.utmx, aq.utmy, s=3,marker='o', c='gray', alpha=.5, label='Sequenza Aquila 2009')
ax.scatter(mainev_ref.utmx, mainev_ref.utmy,s=100, c='r',marker='*', label='Mainshock')


plt.legend(fontsize=11)
plt.xlabel('x (km)')
plt.ylabel('y (km)')
ax=plt.gca()
ax.set_aspect('equal')
ax.grid()

### All we have done so far is nice but ... 
This representation is very limiting for understanding seismicity patterns and characterize the seismic sequence.   
Often times we look at vertical sections to see how earthquakes are distributed as a function of depth.  
This helps in understanding seismogenic faults and fault zone architecture.   

<img src="Figure_1.png" alt="Drawing" width="700"><br>
ref: Valoroso et al., 2014 Geology

## Now let's start to  work on how to plot in section

### [Distanza di un punto da un piano](https://www.youtube.com/watch?v=zWMTTRJ0l4w&list=LL&index=1)

### Let's code the theory we have just seen 
Create a function that allows us to calculate the distance of a point from a plane 

In [ ]:
 # questo ci da la distanza di un punto da un piano generico: ax + by + bc + d = 0 
def distance_point_from_plane(x, y, z, normal, origin):
    # define equation of a plane
    d = -normal[0]*origin[0]-normal[1]*origin[1]-normal[2]*origin[2]
    # solve numerator
    dn = np.abs(normal[0]*x+normal[1]*y+normal[2]*z+d)
    # solve the equation
    dist = dn/np.sqrt(normal[0]**2+normal[1]**2+normal[2]**2)
    # return as output the values of the distances from the plane
    return dist                                                      

Now we have to find the direction of the section that we want to make and it's normal. We will constrain the normal direction using the origin point of the mainshock.

In [ ]:
strike=50    #direction of the section

#normale alla sezione e angolo in radianti
normal_ref=[np.cos(strike*np.pi/180),-np.sin(strike*np.pi/180),0]  

# distanza dalla sezione di tutti i terremoti del catalogo
aq['dist_ref']=distance_point_from_plane(aq.utmx,aq.utmy, -aq['dep'], normal_ref, mainev_ref[['utmx', 'utmy', 'dep']])


At this point we need to find the position along the new x-axis of all the points along the section with azimuth=$\alpha$

In [ ]:
# distances on sections dataset e mainshock

# deactivate pandas property 
pd.options.mode.chained_assignment = None  # default='warn'

# create a new column to store the distance on section for all sequence
aq['X_onsection']=+(aq.utmy-mainev_ref.utmy)*normal_ref[0]-(aq.utmx-mainev_ref.utmx)*normal_ref[1]
 
# Mainshock 
mainev_ref['X_onsection']=+(mainev_ref.utmy-mainev_ref.utmy)*normal_ref[0]-(mainev_ref.utmx-mainev_ref.utmx)*normal_ref[1]


### Plot all the earthquakes on section 

In [ ]:
fig = plt.figure(figsize=(10,10))
# all sequence
plt.scatter(aq.X_onsection, -aq.dep, c='gray',s=3,marker='.',alpha=.2,label='Aquila 2009')

plt.scatter(mainev_ref['X_onsection'], -mainev_ref.dep, marker='*', c='r',s=100, label='Mainshock Mw = 6.0')

plt.ylim(-15,0)
plt.xlim(-10,10)
plt.legend(loc='upper left')

## Let's further analyze the sequence  
I want to see all the earthquakes that take place at a distance less than 1km from the mainshock 

In [ ]:
# selezione dei terremoti con distanza prestabilita [km] dal mainshock
eq_less_1km = aq.loc[np.where(aq.dist_ref <1)]

# selected < 1km
eq_less_1km['X_onsection']=+(eq_less_1km.utmy-mainev_ref.utmy)*normal_ref[0]-(eq_less_1km.utmx-mainev_ref.utmx)*normal_ref[1]
eq_less_1km

### Now we can plot them on the map 


In [ ]:
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111)

ax.scatter(aq.utmx, aq.utmy, s=3,marker='o', c='gray', alpha=.5, label='Sequenza Aquila 2009')
ax.scatter(eq_less_1km.utmx, eq_less_1km.utmy, s=3,marker='o', c='orange', alpha=.5, label='selected Eq 1km')

ax.scatter(mainev_ref.utmx, mainev_ref.utmy,s=100, c='r',marker='*', label='Mainshock')

plt.legend(fontsize=11)
plt.xlabel('x (km)')
plt.ylabel('y (km)')
ax=plt.gca()
ax.set_aspect('equal')
ax.grid()

### Plot on section

In [ ]:
%matplotlib qt

fig = plt.figure(figsize=(10,10))
# all sequence
plt.scatter(aq.X_onsection, -aq.dep, c='gray',s=3,marker='.',alpha=.2,label='Aquila 2009')
# less 1km
plt.scatter(eq_less_1km['X_onsection'],-eq_less_1km['dep'],c='orange',s=20,marker='.',alpha=.5,label='less 1km')
plt.scatter(mainev_ref['X_onsection'], -mainev_ref.dep, marker='*', c='r',s=100, label='Mainshock Mw = 6.0')

plt.ylim(-15,0)
plt.xlim(-10,10)
plt.legend(loc='upper left')


## We can play with the dataset to gather more information about the seismic sequence   
First we can plot together the map and section view

In [ ]:
# plot map and section view 

f, ax = plt.subplots(1,2, figsize=(15,8))

ax[0].set_title('Map view')
ax[0].scatter(aq.utmx, aq.utmy, s=3,marker='o', c='gray', alpha=.5, label='Sequenza Aquila 2009')
ax[0].scatter(eq_less_1km.utmx, eq_less_1km.utmy, s=3,marker='o', c='orange', alpha=.5, label='selected Eq')
ax[0].scatter(mainev_ref.utmx, mainev_ref.utmy,s=100, c='r',marker='*', label='Mainshock')
ax[0].legend(loc='upper left')

ax[1].set_title('Section view')
ax[1].scatter(aq.X_onsection, -aq.dep, c='gray',s=3,marker='.',alpha=.2,label='Aquila 2009')
ax[1].scatter(eq_less_1km.X_onsection,-eq_less_1km.dep,c='orange',s=3,marker='.',alpha=.5,label='Less 1km')
ax[1].scatter(mainev_ref.X_onsection, -mainev_ref.dep, marker='*', c='r',s=100, label='Mainshock Mw = 6.0')
ax[1].set_ylim(-15,0)
ax[1].set_xlim(-10,10)
ax[1].legend(loc='upper left')
f.tight_layout()

### We can make a step further 
Another useful thing that we can do is to plot the seismic sequence in the time domain as well to understand the time-space distribution of the earthquakes

In [ ]:
# plot space time evolution of the sequence 

fig=plt.figure(figsize=(10,10))
plt.title('time-space view')
plt.scatter(aq.time, aq.X_onsection,c='gray',s=3,alpha=0.2)
plt.scatter(eq_less_1km.time, eq_less_1km.X_onsection, c='orange',s=20,marker='.',alpha=.5,label='Less 1km')
plt.scatter(mainev_ref.time, mainev_ref.X_onsection, c='r', s=150, marker='*')
plt.ylim(20,-20)
plt.xlabel('Date')
plt.ylabel('distance from the mainshock [km]')

### Now we can unify all the three views

In [ ]:
# put the three views all together 

f, ax = plt.subplots(1,3, figsize=(20,8))

ax[0].set_title('Map view')
ax[0].scatter(aq.utmx, aq.utmy, s=3,marker='o', c='gray', alpha=.5, label='Sequenza Aquila 2009')
ax[0].scatter(eq_less_1km.utmx, eq_less_1km.utmy, s=5,marker='o', c='orange', alpha=.5, label='selected Eq')
ax[0].scatter(mainev_ref.utmx, mainev_ref.utmy,s=100, c='r',marker='*', label='Mainshock')
ax[0].legend(loc='upper left')

ax[1].set_title('Section view')
ax[1].scatter(aq.X_onsection, -aq.dep, c='gray',s=3,marker='.',alpha=.2,label='Aquila 2009')
ax[1].scatter(eq_less_1km.X_onsection,-eq_less_1km.dep,c='orange',s=3,marker='.',alpha=.5,label='Less 1km')
ax[1].scatter(mainev_ref.X_onsection, -mainev_ref.dep, marker='*', c='r',s=100, label='Mainshock Mw = 6.0')
ax[1].set_ylim(-15,0)
ax[1].set_xlim(-10,10)
ax[1].legend(loc='upper left')

ax[2].set_title('time-space view')
ax[2].scatter(aq.time, aq.X_onsection,c='gray',s=3,alpha=0.2)
ax[2].scatter(eq_less_1km.time, eq_less_1km.X_onsection, c='orange',s=20,marker='.',alpha=.5,label='Less 1km')
ax[2].scatter(mainev_ref.time, mainev_ref.X_onsection, c='r', s=150, marker='*')
ax[2].set_ylim(20,-20)

f.tight_layout()

#### However, we do not have any information about the magnitude of the earthquakes   
We can choose to select all the earthquakes with a magnitude in a prescribed range. 

In [ ]:
# highlight eq>M3 within the 1km distance from the fault
M3_less_1km = eq_less_1km[(eq_less_1km.mag>3) & (eq_less_1km.mag<5.8)]


f, ax = plt.subplots(1,3, figsize=(20,8))

ax[0].set_title('Map view')
ax[0].scatter(aq.utmx, aq.utmy, s=3,marker='o', c='gray', alpha=.5, label='Sequenza Aquila 2009')
ax[0].scatter(eq_less_1km.utmx, eq_less_1km.utmy, s=5,marker='o', c='orange', alpha=.5, label='selected Eq')
ax[0].scatter(mainev_ref.utmx, mainev_ref.utmy,s=100, c='r',marker='*', label='Mainshock')
ax[0].scatter(M3_less_1km.utmx, M3_less_1km.utmy,s=30, c='b',marker='D', label='M>3')
ax[0].legend(loc='lower left')

ax[1].set_title('Section view')
ax[1].scatter(aq.X_onsection, -aq.dep, c='gray',s=3,marker='.',alpha=.2,label='Aquila 2009')
ax[1].scatter(eq_less_1km.X_onsection,-eq_less_1km.dep,c='orange',s=3,marker='.',alpha=.5,label='Less 1km')
ax[1].scatter(mainev_ref.X_onsection, -mainev_ref.dep, marker='*', c='r',s=100, label='Mainshock Mw = 6.0')
ax[1].scatter(M3_less_1km.X_onsection, -M3_less_1km.dep,s=30, c='b',marker='D', label='M>3')

ax[1].set_ylim(-15,0)
ax[1].set_xlim(-10,10)

ax[2].set_title('time-space view')
ax[2].scatter(aq.time, aq.X_onsection,c='gray',s=3,alpha=0.2)
ax[2].scatter(eq_less_1km.time, eq_less_1km.X_onsection, c='orange',s=20,marker='.',alpha=.5,label='Less 1km')
ax[2].scatter(mainev_ref.time, mainev_ref.X_onsection, c='r', s=150, marker='*')
ax[2].scatter(M3_less_1km.time, M3_less_1km.X_onsection,s=30, c='b',marker='D', label='M>3')

ax[2].set_ylim(20,-20)

f.tight_layout()



## Some extra plots that may or may not work

In [ ]:
from datetime import datetime
import matplotlib.dates as mdates

eq_M2 = np.where(aq.mag>2)

# set a variable for dates
dates = aq.time.iloc[eq_M2]
# normalize eq 
eq_norm = (aq.mag.iloc[eq_M2]-min(aq.mag.iloc[eq_M2]))/(max(aq.mag.iloc[eq_M2])-min(aq.mag.iloc[eq_M2]))

cm = plt.cm.get_cmap('plasma')

# make the plot
f, ax = plt.subplots(1,1, figsize=(8,8))

sc = ax.scatter(aq.utmx.iloc[eq_M2], aq.utmy.iloc[eq_M2], c=mdates.date2num(dates),  marker='o', cmap=cm, s=eq_norm*150)
sc = f.colorbar(mappable=sc, ax=ax)
sc.set_label('Time')
loc = mdates.AutoDateLocator()
sc.ax.yaxis.set_major_locator(loc)
sc.ax.yaxis.set_major_formatter(mdates.ConciseDateFormatter(loc))

ax.scatter(aq.utmx, aq.utmy, s=3,marker='o', c='gray', alpha=.2, label='Sequenza Aquila 2009', zorder=0)
ax.scatter(mainev_ref.utmx, mainev_ref.utmy,s=150, c='r',marker='*', label='Mainshock')

ax.legend(fontsize=11)
ax.set_xlabel('x (km)')
ax.set_ylabel('y (km)')
ax=plt.gca()
ax.set_aspect('equal')
ax.grid(zorder=0)
f.tight_layout()

In [ ]:
# where are eq > 3 for the selected <1km portion?
eq_g_M3 = eq_less_1km.loc[eq_less_1km.mag>2]

cm = plt.cm.get_cmap('plasma')

f, ax = plt.subplots(1,1, figsize=(10,10))
sc = ax.scatter(aq.utmx.iloc[eq_g_M3], aq.utmy.iloc[eq_g_M3], c=aq.mag.iloc[eq_g_M3], vmin=min(aq.mag.iloc[eq_g_M3]), vmax=max(aq.mag.iloc[eq_g_M3]), cmap=cm)
sc = f.colorbar(sc)
sc.set_label('Magnitude')

ax.scatter(aq.utmx, aq.utmy, s=3,marker='o', c='gray', alpha=.2, label='Sequenza Aquila 2009', zorder=0)
ax.scatter(mainev_ref.utmx, mainev_ref.utmy,s=150, c='r',marker='*', label='Mainshock')

ax.legend(fontsize=11)
ax.set_xlabel('x (km)')
ax.set_ylabel('y (km)')
ax=plt.gca()
ax.set_aspect('equal')
ax.grid()
f.tight_layout()

f, ax = plt.subplots(1,2, figsize=(15,8))

ax[0].set_title('Map view')
ax[0].scatter(aq.utmx, aq.utmy, s=3,marker='o', c='gray', alpha=.5, label='Sequenza Aquila 2009')
ax[0].scatter(eq_less_1km.utmx, eq_less_1km.utmy, s=5,marker='o', c='orange', alpha=.5, label='selected Eq')
ax[0].scatter(mainev_ref.utmx, mainev_ref.utmy,s=100, c='r',marker='*', label='Mainshock')
ax[0].legend(loc='upper left')

ax[1].set_title('Section view')
ax[1].scatter(aq.X_onsection, -aq.dep, c='gray',s=3,marker='.',alpha=.2,label='Aquila 2009')
ax[1].scatter(eq_less_1km.X_onsection,-eq_less_1km.dep,c='orange',s=3,marker='.',alpha=.5,label='Less 1km')
ax[1].scatter(mainev_ref.X_onsection, -mainev_ref.dep, marker='*', c='r',s=100, label='Mainshock Mw = 6.0')
ax[1].set_ylim(-15,0)
ax[1].set_xlim(-10,10)
ax[1].legend(loc='upper left')
f.tight_layout()

### Now let's play on section 

In [ ]:
# earthquake with magnitude > 3
M3 = eq_less_1km.iloc[np.where(eq_less_1km.mag>3)]

f, ax = plt.subplots(1,1, figsize=(8,10))
ax.scatter(eq_less_1km.X_onsection, -eq_less_1km.dep, c='gray', s=5, alpha=.5)
sc = ax.scatter(M3.X_onsection, -M3.dep, c=M3.mag, vmin=min(M3.mag), vmax=max(M3.mag))

plt.scatter(mainev_ref['X_onsection'], -mainev_ref.dep, marker='*', c='r',s=100, label='Mainshock Mw = 6.0')

plt.ylim(-15,0)
plt.xlim(-10,10)
plt.legend(loc='upper left')

In [ ]:
# colorbar by magnitude 
M3 = eq_less_1km.iloc[np.where(eq_less_1km.mag>3)]
cm = plt.cm.get_cmap('plasma')

f, ax = plt.subplots(1,1, figsize=(8,10))
ax.scatter(eq_less_1km.X_onsection, -eq_less_1km.dep, c='gray', s=5, alpha=.5)
ax.scatter(M3.X_onsection, -M3.dep, c='g')

plt.scatter(mainev_ref['X_onsection'], -mainev_ref.dep, marker='*', c='r',s=100, label='Mainshock Mw = 6.0')

plt.ylim(-15,0)
plt.xlim(-10,10)
plt.legend(loc='upper left')

In [ ]:
%matplotlib qt
import matplotlib.colors as mcolors



timediff = eq_less_1km.time-mainev_ref.time
timediff = mdates.date2num(timediff)

cm = plt.cm.get_cmap('seismic')

vcenter = 0
vmin, vmax = timediff.min(), timediff.max()
normalize = mcolors.TwoSlopeNorm(vcenter=vcenter, vmin=vmin, vmax=vmax)


f, ax = plt.subplots(1,1,figsize=(10,10))
sc = ax.scatter(eq_less_1km.X_onsection,-eq_less_1km.dep,c=-timediff, norm=normalize,   
                marker='o', cmap=cm,alpha=.5,label='Aquila 2009')
sc = f.colorbar(mappable=sc, ax=ax)
sc.set_label('Time')
# loc = mdates.AutoDateLocator()
# sc.ax.yaxis.set_major_locator(loc)
# sc.ax.yaxis.set_major_formatter(mdates.ConciseDateFormatter(loc))


ax.scatter(mainev_ref.X_onsection, -mainev_ref.dep, marker='*', c='r',s=100, label='Mainshock Mw = 6.0')

plt.ylim(-15,0)
plt.xlim(-10,10)
plt.legend(loc='best')



In [ ]:
fig = plt.figure(figsize=(10,10))
plt.scatter(X_onsection_1km,-aq['dep'].loc[eq_less_1km],c='orange',s=3,marker='.',alpha=.5,label='<1km')
plt.scatter(X_onsection_05km,-aq['dep'].loc[eq_less_05km],c='g',s=3,marker='.',alpha=.5,label='<0.5km')

plt.scatter(X_onsection_mainevent, -mainev_ref.dep, marker='*', c='r',s=100, label='Mainshock Mw = 6.0')

plt.ylim(-15,0)
plt.xlim(-10,10)
plt.legend(loc='upper left')